In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm, ksone
from testes_kolmogorov1 import amostra_normal, amostra_gamma, amostra_quiquadrada


# Teste de kolmogorov-smirnov

In [3]:
def kolmogorov_smirnov(caminho_do_arquivo, alpha):
    # Lendo o arquivo csv
    df = pd.read_csv(caminho_do_arquivo)
    for i in range(len(df.columns) + 1):
        lista_de_valores = df.iloc[:, i].values
        # Obtendo a média amostral
        media_amostral = np.mean(lista_de_valores)
        # Obtendo o desvio padrão amostral
        desvio_padrao_amostral = np.std(lista_de_valores, ddof=1)
        # Ordenando a lista de valores sem valores repetidos
        lista_ordenada = sorted(lista_de_valores)
        Xi = sorted(set(lista_ordenada))
        # Calculando a frequência absoluta de cada valor
        freq_abs = [lista_ordenada.count(i) for i in Xi]
        # Calculando a frequência acumulada
        freq_acum = []
        for i in range(len(freq_abs)):
            if i == 0:
                freq_acum.append(freq_abs[i])
            else:
                freq_acum.append(freq_acum[i-1] + freq_abs[i])
        # Calculando a frequência relativa acumulada
        freq_rel_acum = []
        for i in range(len(freq_abs)):
                freq_rel_acum.append(freq_acum[i]/freq_acum[-1])
        # Calculandos os valores de Zi
        Zi = [(x - media_amostral) / desvio_padrao_amostral for x in Xi]
        # Calculando a frequência esperada Fesp (valor de Z na tabela de distribuição normal N(0,1))
        freq_esperada = [norm.cdf(z) for z in Zi] #Funçao de distribuição acumulada da normal
        # Calculando a |Fesp(Xi) - Frac(Xi)| para cada valor de Xi
        Fesp_menos_Frac = [abs(freq_esperada[i] - freq_rel_acum[i]) for i in range(len(Xi))]
        # Calculando a |Fesp(Xi) - Frac(Xi-1)| para cada valor de Xi
        # Como o for não pode começar em 0, então a primeira posição é calculada fora do for
        Fesp_menos_Frac_1 = [freq_esperada[0]] + [abs(freq_esperada[i] - freq_rel_acum[i-1]) for i in range(1, len(Xi))]
        #Calculando o Dcalc
        max_Fesp_menos_Frac = max(Fesp_menos_Frac)
        max_Fesp_menos_Frac_1 = max(Fesp_menos_Frac_1)
        Dcalc = max(max_Fesp_menos_Frac, max_Fesp_menos_Frac_1)

        #Calculando o Dtabelado (D crítico)
        if len(lista_ordenada) > 0 and len(lista_ordenada) <= 35:
            Dtab = ksone.ppf(1 - alpha/2, len(lista_ordenada)) #Calcula o Dcrit de um teste KS
        
        elif len(lista_ordenada) > 35:
            if alpha == 0.2:
                Dtab = 1.07 / np.sqrt(len(lista_ordenada))
            elif alpha == 0.15:
                Dtab = 1.14 / np.sqrt(len(lista_ordenada))
            elif alpha == 0.10:
                Dtab = 1.22 / np.sqrt(len(lista_ordenada))
            elif alpha == 0.05:
                Dtab = 1.36 / np.sqrt(len(lista_ordenada))
            elif alpha == 0.01:
                Dtab = 1.63 / np.sqrt(len(lista_ordenada))
        
        resultado = []
        if Dcalc < Dtab:
            print("Dcalc = %.4f < Dtab = %.4f" % (Dcalc, Dtab))
            print("Aceito a hipótese H0 que a amostra segue uma distribuição normal.")
            resultado.append(1)
        else:
            print("Dcalc = %.4f > Dtab = %.4f" % (Dcalc, Dtab))
            print("Rejeito a hipótese H0 que a amostra segue uma distribuição normal")
            resultado.append(0)
        
        # Criando um dataframe no pandas com todas as colunas das frequencias

        #df = pd.DataFrame({'Xi': Xi, 'FreqAbs': freq_abs, 'FreqAcum': freq_acum, 'FreqRelAcum': freq_rel_acum, 'Zi': Zi, 'Fesp': freq_esperada, '|Fesp - Frac|': Fesp_menos_Frac, '|Fesp - Frac-1|': Fesp_menos_Frac_1})
        #return df
        return resultado


In [4]:
def kolmogorov_smirnov_(caminho_do_arquivo, alpha):
    # Lendo o arquivo csv
    df = pd.read_csv(caminho_do_arquivo)
    # Obtendo a coluna de valores
    for i in range(len(df.columns) + 1):
        lista_de_valores = df.iloc[:, i].values
        # Ordenando a lista de valores sem valores repetidos
        lista_ordenada = sorted(lista_de_valores)
        #Obtendo a amplitude total e a amplitude de cada classe
        k = len(lista_ordenada) // 3 # quantidade de classes
        amplitude_total = max(lista_ordenada) - min(lista_ordenada)
        amplitude_classe = amplitude_total /k
        # Obtendo os limites de cada classe
        limites_classes = [min(lista_ordenada) + i*amplitude_classe for i in range(k + 1)]
        # Colocando os limites de classe na coluna Xi
        Xi = [limites_classes[i] for i in range(k)]
        # Calculando a média dos intervalos
        media_amostral = np.mean(Xi)
        # Calculando o desvio padrao dos intervalos
        desvio_padrao_amostral = np.std(Xi, ddof=1)
        # Calculando a frequência absoluta de cada classe
        freq_abs = []
        for i in range(k):
            freq_abs.append(len([x for x in lista_ordenada if x >= limites_classes[i] and x < limites_classes[i+1]]))

        # Calculando a frequência acumulada
        freq_acum = []
        for i in range(len(freq_abs)):
            if i == 0:
                freq_acum.append(freq_abs[i])
            else:
                freq_acum.append(freq_acum[i-1] + freq_abs[i])
        # Calculando a frequência relativa acumulada
        freq_rel_acum = []
        for i in range(len(freq_abs)):
                freq_rel_acum.append(freq_acum[i]/freq_acum[-1])
        # Calculandos os valores de Zi
        Zi = [(x - media_amostral) / desvio_padrao_amostral for x in Xi]
        # Calculando a frequência esperada Fesp (valor de Z na tabela de distribuição normal N(0,1))
        freq_esperada = [norm.cdf(z) for z in Zi] #Funçao de distribuição acumulada da normal
        # Calculando a |Fesp(Xi) - Frac(Xi)| para cada valor de Xi
        Fesp_menos_Frac = [abs(freq_esperada[i] - freq_rel_acum[i]) for i in range(len(Xi))]
        # Calculando a |Fesp(Xi) - Frac(Xi-1)| para cada valor de Xi
        # Como o for não pode começar em 0, então a primeira posição é calculada fora do for
        Fesp_menos_Frac_1 = [freq_esperada[0]] + [abs(freq_esperada[i] - freq_rel_acum[i-1]) for i in range(1, len(Xi))]
        #Calculando o Dcalc
        max_Fesp_menos_Frac = max(Fesp_menos_Frac)
        max_Fesp_menos_Frac_1 = max(Fesp_menos_Frac_1)
        Dcalc = max(max_Fesp_menos_Frac, max_Fesp_menos_Frac_1)

        #Calculando o Dtabelado (D crítico)
        if len(lista_ordenada) > 0 and len(lista_ordenada) <= 35:
            Dtab = ksone.ppf(1 - alpha/2, len(lista_ordenada)) #Calcula o Dcrit de um teste KS
        
        elif len(lista_ordenada) > 35:
            if alpha == 0.2:
                Dtab = 1.07 / np.sqrt(len(lista_ordenada))
            elif alpha == 0.15:
                Dtab = 1.14 / np.sqrt(len(lista_ordenada))
            elif alpha == 0.10:
                Dtab = 1.22 / np.sqrt(len(lista_ordenada))
            elif alpha == 0.05:
                Dtab = 1.36 / np.sqrt(len(lista_ordenada))
            elif alpha == 0.01:
                Dtab = 1.63 / np.sqrt(len(lista_ordenada))
        
        resultado = []
        if Dcalc < Dtab:
            print("Dcalc = %.4f < Dtab = %.4f" % (Dcalc, Dtab))
            print("Aceito a hipótese H0 que a amostra segue uma distribuição normal.")
            resultado.append(1)
        else:
            print("Dcalc = %.4f > Dtab = %.4f" % (Dcalc, Dtab))
            print("Rejeito a hipótese H0 que a amostra segue uma distribuição normal")
            resultado.append(0)
        
        # Criando um dataframe no pandas com todas as colunas das frequencias

        #df = pd.DataFrame({'Xi': Xi, 'FreqAbs': freq_abs, 'FreqAcum': freq_acum, 'FreqRelAcum': freq_rel_acum, 'Zi': Zi, 'Fesp': freq_esperada, '|Fesp - Frac|': Fesp_menos_Frac, '|Fesp - Frac-1|': Fesp_menos_Frac_1})
        #return df
        return resultado

In [5]:
valores = [52, 50, 36, 40, 30, 42, 38, 38, 52, 44, 36, 34, 50, 42, 34, 55, 36, 55, 42, 52, 34, 48, 55, 
           44, 44, 30, 48, 40, 40, 44, 40, 44, 38, 36, 50, 42]
alpha = 0.05
kolmogorov_smirnov(valores, alpha)

Dcalc = 0.1184 < Dtab = 0.2267
Aceito a hipótese H0 que a amostra segue uma distribuição normal.


[1]

In [6]:
conjunto_de_amostras_normais = amostra_normal()
resultados = []
for amostra in conjunto_de_amostras_normais:
    resultados.append(kolmogorov_smirnov(amostra, alpha))
print(resultados)

Dcalc = 0.0912 < Dtab = 0.3489
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.2084 < Dtab = 0.3273
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.0932 < Dtab = 0.2544
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.1623 < Dtab = 0.3014
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.1491 < Dtab = 0.2236
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.1401 < Dtab = 0.2499
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.1528 < Dtab = 0.2308
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.0941 < Dtab = 0.2150
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.1281 < Dtab = 0.2941
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.1055 < Dtab = 0.2242
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.2306 < Dtab = 0.3489

In [7]:
resultados = []
conjunto_de_amostras_gama = amostra_gamma()
for amostra in conjunto_de_amostras_gama:
    resultados.append(kolmogorov_smirnov_(amostra, alpha))
print(resultados)

Dcalc = 0.3500 > Dtab = 0.2872
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.5370 > Dtab = 0.2809
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.3388 > Dtab = 0.2457
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.5045 > Dtab = 0.2749
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.3422 > Dtab = 0.3014
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.4357 > Dtab = 0.3094
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.3385 > Dtab = 0.2693
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.2230 < Dtab = 0.2236
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.2299 < Dtab = 0.2640
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.3105 > Dtab = 0.2417
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
[[0], [0], [0], [0], [0], [0],

In [8]:
resultados = []
conjunto_de_amostras_quiquad = amostra_quiquadrada()
for amostra in conjunto_de_amostras_quiquad:
    resultados.append(kolmogorov_smirnov_(amostra, alpha))
print(resultados)

Dcalc = 0.4553 > Dtab = 0.3014
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.2289 < Dtab = 0.2544
Aceito a hipótese H0 que a amostra segue uma distribuição normal.
Dcalc = 0.4510 > Dtab = 0.2749
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.6987 > Dtab = 0.2941
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.7318 > Dtab = 0.2206
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.6409 > Dtab = 0.2749
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.5646 > Dtab = 0.2178
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.3622 > Dtab = 0.2379
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.7709 > Dtab = 0.2150
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
Dcalc = 0.3538 > Dtab = 0.2242
Rejeito a hipótese H0 que a amostra segue uma distribuição normal
[[0], [1], [0], [0], [0], [0],